In [9]:
# -*- coding: utf-8 -*-
"""
Created on Tue Jul  2 10:16:16 2024

@author: abhis
"""

import os
import sys

file_dir = os.path.dirname(os.getcwd())
sys.path.append(file_dir) 
import pandas as pd
from neo4j import GraphDatabase
import os
import requests
import json
import requests
import numpy as np
from graphdatascience import GraphDataScience
wd = os.getcwd()
from dotenv import load_dotenv
import graph_build,create_plot_embeddings
from modules import plot_vector_search,qa_bot,visualiser
import openai
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.agent import AgentFinish
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.agents import AgentExecutor


load_dotenv()

#openai.api_key = os.environ['OPENAI_API_KEY']
database = os.getenv('NEO4J_DATABASE')
uri, user, password = os.getenv('NEO4J_URI'), os.getenv('NEO4J_USER'),'@PromptEngg'
gds = GraphDataScience(
    uri,
    auth = (user, password), database=database
)       

def graph_init():
     '''
     One time process - Incase starting from scratch use this to create graph, and embeddings for plot
     '''
     print("Graph and Embeddings already Created - Check modules.utils to enable")
     #graph_build.create_logistics_graph()
     #create_plot_embeddings.create_plot_embeddings()
    
def chat_bot(query):
    '''
    Chatbot with functions to answer questions related to the movies
    '''
    tools=[plot_vector_search.vectorSearch,qa_bot.chat,visualiser.visualise]
    functions = [format_tool_to_openai_function(f) for f in tools]
    model = ChatOpenAI(temperature=0).bind(functions=functions)
    memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history",run_intermediate_steps=True,return_direct=True)

    promptengg = """You are a helpful assistant specialized in providing insights and information related to supply chains, specifically focusing on the SupplyChainInsights application. Your expertise includes understanding and explaining the intricacies of supply chain data, including commodity shipments, pricing trends, and logistics. You should only respond to queries related to:

                Analyzing trends and ranges in the pricing of health commodities.
                Understanding the volume and movement of Antiretroviral (ARV) and HIV lab shipments.
                Insights into the supply chain logistics, including costs and lead times by product and country.
                Comparing and contrasting data from the SupplyChainInsights application with the Global Fund's Price, Quality, and Reporting (PQR) data.
                Any other questions related to understanding supply chain concepts, data, and insights within the context of the provided dataset.
                Please refrain from discussing topics outside of these areas. Your responses should be concise, informative, and tailored to help users make better data-driven decisions regarding supply chain management."""

    prompt = ChatPromptTemplate.from_messages([
        ("system", promptengg),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ])
    chain = RunnablePassthrough.assign(
        agent_scratchpad = lambda x: format_to_openai_functions(x["intermediate_steps"])
    ) | prompt | model | OpenAIFunctionsAgentOutputParser()
    qa = AgentExecutor(agent=chain, tools=tools, verbose=True, memory=memory,return_intermediate_steps=True)
    result = qa.invoke({"input": query})
    if result['intermediate_steps'][0][0].tool=='visualise':
        answer = result['intermediate_steps'][0][-1]
        answer['Tool'] = 'visualise'
    else:
        answer = result['output'] 
    return answer   


a = chat_bot("Give me a Line chart of the number of brands by each vendor?")
print(a)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated field from a procedure. ('advertisedListenAddress' returned by 'gds.debug.arrow' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: 'CALL gds.debug.arrow()'
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated field from a procedure. ('serverLocation' returned by 'gds.debug.arrow' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: 'CALL gds.debug.arrow()'




> Entering new AgentExecutor chain...

Invoking: `chat` with `{'que': 'Number of brands by each vendor'}`




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (v:Vendor)-[:VENDOR]->(p:Product)-[:BRAND]->(b:Brand) 
WITH v.name as Vendor, count(DISTINCT b) as NumberOfBrands 
RETURN Vendor, NumberOfBrands

> Finished chain.


C:\Users\abhis\AppData\Local\Continuum\anaconda3\envs\streamlit\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


- MEDMIRA EAST AFRICA LTD.: 1 brand
- RANBAXY Fine Chemicals LTD.: 1 brand
- CIPLA LIMITED: 1 brand
- SCMS from RDC: 24 brands
- Aurobindo Pharma Limited: 1 brand
- HETERO LABS LIMITED: 1 brand
- STRIDES ARCOLAB LIMITED: 1 brand
- PHARMACY DIRECT: 10 brands
- MICRO LABS LIMITED: 1 brand
- EMCURE PHARMACEUTICALS LTD: 1 brandHere is the number of brands by each vendor:
- MEDMIRA EAST AFRICA LTD.: 1 brand
- RANBAXY Fine Chemicals LTD.: 1 brand
- CIPLA LIMITED: 1 brand
- SCMS from RDC: 24 brands
- Aurobindo Pharma Limited: 1 brand
- HETERO LABS LIMITED: 1 brand
- STRIDES ARCOLAB LIMITED: 1 brand
- PHARMACY DIRECT: 10 brands
- MICRO LABS LIMITED: 1 brand
- EMCURE PHARMACEUTICALS LTD: 1 brand

> Finished chain.
{'input': 'Give me the number of brands by each vendor?', 'chat_history': [HumanMessage(content='Give me the number of brands by each vendor?'), AIMessage(content='Here is the number of brands by each vendor:\n- MEDMIRA EAST AFRICA LTD.: 1 brand\n- RANBAXY Fine Chemicals LTD.: 1 brand

In [12]:
type(a)==dict

True

In [6]:
def display_result(result_text):
    # Define a regular expression pattern to detect a file path
    path_pattern = r'([a-zA-Z]:\\[\\a-zA-Z0-9._-]+)'

    # Search for the path in the result_text
    match = re.search(path_pattern, result_text)

    if match:
        image_path = match.group(1)
        print(image_path)
        if os.path.exists(image_path):
            # Display the image using PIL or another appropriate method
            image = Image.open(image_path)
            image.show()
            # Alternatively, display using matplotlib
            plt.imshow(image)
            plt.axis('off')
            plt.show()
        else:
            print("Image file not found at the specified path.")
    else:
        print(result_text)
import re
display_result(a['output'])

C:\Users\abhis\Desktop\NEU\INFO7375
Image file not found at the specified path.
